In [1]:
import tkinter as tk

In [2]:
class CurveGrid(tk.Canvas):
    def __init__(self, master, **kwargs):
        super().__init__(master, **kwargs)

        # left button
        self.bind("<Button-1>", self.on_click)
        # click and hold
        self.bind("<B1-Motion>", self.on_drag)
        # left mouse button release
        self.bind("<ButtonRelease-1>", self.on_release)
        self.points = [(50, 510), (150, 200), (250, 100), (510, 50)]
        self.lines = []

        self.draw_grid()

    def draw_grid(self):
        self.delete("all")

        # vertical lines
        for x in range(50, 511, 92):
            self.create_line(x, 50, x, 511)

        # horizontal lines
        for y in range(50, 511, 92):
            self.create_line(50, y, 511, y)

        # def draw_bezier(p0, p1, p2):
        #     t = np.linspace(0, 1, 100)
        #     points = (1-t)**2*p0 + 2*(1-t)*t*p1 + t**2*p2
        #     for i in range(len(points)-1):
        #         canvas.create_line(points[i], points[i+1], smooth=True)

        # curve
        self.lines = []
        for i in range(len(self.points) - 1):
            self.lines.append(
                self.create_line(
                    self.points[i], self.points[i+1], fill="red", width=2, smooth=True
                )
            )

        # points
        for x, y in self.points:
            self.create_oval(x-5, y-5, x+5, y+5, fill="white", outline="black")

    def on_click(self, event):
        x, y = event.x, event.y
        index = min(range(
            len(self.points)),
            key=lambda i: (self.points[i][0]-x)**2 + (self.points[i][1]-y)**2
        )
        self.selected_index = index

    # drag point
    def on_drag(self, event):
        x, y = event.x, event.y
        x = max(50, min(x, 510))
        y = max(50, min(y, 510))
        self.points[self.selected_index] = (x, y)
        self.draw_grid()

    def on_release(self, event):
        self.selected_index = None

In [3]:
root = tk.Tk()
canvas = CurveGrid(root, width=560, height=560)
canvas.pack()

root.resizable(True, True)
root.title('сurves')
# root.geometry("400x400")

# label = tk.Label(root, text="hello world!")
# label.pack()
#
# button = tk.Button(root, text="click")
# button.pack()

In [4]:
root.mainloop()

In [5]:
# import tkinter as tk
# import numpy as np
#
# # Создаем холст
# root = tk.Tk()
# canvas = tk.Canvas(root, width=400, height=400)
# canvas.pack()
#
# # Определяем начальные точки контроля
# p0 = np.array([100, 100])
# p1 = np.array([200, 200])
# p2 = np.array([300, 100])
#
# # Определяем функцию, которая рисует кривую Безье с помощью заданных точек контроля
# def draw_bezier(p0, p1, p2):
#     t = np.linspace(0, 1, 100)
#     points = (1-t)**2*p0 + 2*(1-t)*t*p1 + t**2*p2
#     for i in range(len(points)-1):
#         canvas.create_line(points[i], points[i+1], smooth=True)
#
# # Определяем функцию, которая изменяет положение точек контроля и перерисовывает кривую Безье
# def animate():
#     global p0, p1, p2
#     canvas.delete("all")
#     # Изменяем положение точек контроля
#     p0 += np.array([1, 0])
#     p1 += np.array([0, 1])
#     p2 += np.array([-1, 0])
#     # Рисуем кривую Безье с новыми точками контроля
#     draw_bezier(p0, p1, p2)
#     # Запускаем анимацию повторно через 10 миллисекунд
#     root.after(120, animate)
#
# # Запускаем анимацию
# animate()
#
# # Запускаем главный цикл обработки событий
# root.mainloop()


In [6]:
import tkinter as tk

class Point:
    def __init__(self, canvas, x, y):
        self.canvas = canvas
        self.id = canvas.create_oval(x-5, y-5, x+5, y+5, fill="blue")
        self.x = x
        self.y = y
        self.canvas.bind("<B1-Motion>", self.move)

    def move(self, event):
        x, y = event.x, event.y
        # Ограничение перемещения точки в диапазоне [50, 250] по оси x
        x = max(50, min(x, 250))
        # Ограничение перемещения точки в диапазоне [50, 250] по оси y
        y = max(50, min(y, 250))
        self.canvas.coords(self.id, x-5, y-5, x+5, y+5)
        self.x = x
        self.y = y

root = tk.Tk()
canvas = tk.Canvas(root, width=300, height=300)
canvas.pack()

point = Point(canvas, 150, 150)

root.mainloop()